In [1]:
from astropy.table import Table
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from minisom import MiniSom
import pickle

In [2]:
df_L3_info = Table.read('../dataset/L3_COSMOS2020_Richard_RefCat_2023DEC4_info.fits')
df_L3_info = df_L3_info.to_pandas()

In [3]:
L3_phot_data = np.loadtxt('../dataset/L3_COSMOS2020_Richard_RefCat_2023DEC4_averaged_phot102.txt')
phot_data = L3_phot_data[:,3::2]
phot_err_data = L3_phot_data[:,4::2]

In [4]:
data = (phot_data - np.mean(phot_data, axis=0)) / np.std(phot_data, axis=0)

In [5]:
with open("../grid_search/record.pkl", "rb") as fh:
    record = pickle.load(fh)

In [6]:
def grid_search(record, data):
    for i in record:
        if "som" not in i:
            som = MiniSom(i["dim"], i["dim"], data.shape[1], sigma = i["sigma"], learning_rate = i["rate"])
            som.random_weights_init(data)
            som.train(data = data, num_iteration = 1, use_epochs = True)
            i["topo_err"] = som.topographic_error(data)
            i["quan_err"] = som.quantization_error(data)
            i["som"] = som
            print("Start saving file")
            with open("../grid_search/record.pkl", "wb") as fh:
                pickle.dump(record, fh)
            print("Finish savinf file")
        else:
            pass

In [ ]:
grid_search(record, data)

Start saving file
Finish savinf file


D:\miniconda\envs\SPHEREx_SOM\Lib\site-packages\minisom.py:540: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(-2 * cross_term + input_data_sq + weights_flat_sq.T)


Start saving file
Finish savinf file
Start saving file
Finish savinf file
Start saving file
Finish savinf file
Start saving file
Finish savinf file
Start saving file
Finish savinf file
Start saving file
Finish savinf file
Start saving file
Finish savinf file
Start saving file
Finish savinf file
Start saving file
Finish savinf file
Start saving file
Finish savinf file
Start saving file
Finish savinf file
Start saving file
Finish savinf file
